# Transformación de datos en SQLite3


>[Transformación de datos en SQLite3](#scrollTo=yiK9krf4gJty)

>>[Preparación](#scrollTo=940OO7FzgJt0)

>>>[Cellmagics](#scrollTo=5udxtDL_gOxs)

>>[Strings](#scrollTo=WielHa34gJt1)

>>>[LENGTH()](#scrollTo=Cz-axOd0gJt1)

>>>[UPPER(), LOWER()](#scrollTo=2Wsy1HwRgJt2)

>>>[LIKE](#scrollTo=edfgiIcPgJt3)

>>[ORDER BY](#scrollTo=Frz1-afJgJt4)

>>[GROUP BY](#scrollTo=saT3Ko1MgJt4)

>>>[COUNT()](#scrollTo=4Qlo3q5MgJt4)

>>>[MAX(), MIN(), SUM(), AVG()](#scrollTo=YDIRtS9pgJt5)



## Preparación

### Cellmagics

Se crean dos cellmagics. La cellmagic `%%sqlite3` permite ejecutar el contenido de la celda en el motor de búsqueda cuando la celda contiene una sola sentencia; el cellmagic retorna el resultado de la ejecución del comando SQL. 

La cellmagic `%%sqlite3script` permite ejecutar scripts conformados por varios comandos SQL. No retorna resultados

In [29]:
import sqlite3

conn = sqlite3.connect(":memory:")
cur = conn.cursor()

In [30]:
# Intento para que no muestre Warnings
!pip install ipython-sql

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [31]:
# Intento para que no muestre Warnings
%load_ext sql

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [32]:
from IPython.core.magic import register_cell_magic, register_line_magic


@register_cell_magic
def sqlite3(line, cell):
    cell = [
        l for l in cell.split("\n") if l[0:2] != "--" and l[0:2] not in ["--", "##"]
    ]
    cell = "\n".join(cell)
    return cur.execute(cell).fetchall()


@register_cell_magic
def sqlite3script(line, cell):
    cell = [l for l in cell.split("\n") if len(l) >= 2 and l[0:2] not in ["--", "##"]]
    cell = "\n".join(cell)
    conn.executescript(cell)
    conn.commit()


@register_line_magic
def sql(line):
    line = [
        l for l in line.split("\n") if l[0:2] != "--" and l[0:2] not in ["--", "##"]
    ]
    line = "\n".join(line)
    return cur.execute(line).fetchall()

In [35]:
%%sqlite3script

DROP TABLE IF EXISTS persons;

CREATE TABLE persons (
    id INT,
    firstname VARCHAR(10),
    surname VARCHAR(10),
    birthday DATE,
    color VARCHAR(10),
    quantity INT
);

INSERT INTO persons VALUES
    (1,"Vivian","Hamilton","1971-07-08","green",1),
    (2,"Karen","Holcomb","1974-05-23","green",4),
    (3,"Cody","Garrett","1973-04-22","orange",1),
    (4,"Roth","Fry","1975-01-29","black",1),
    (5,"Zoe","Conway","1974-07-03","blue",2),
    (6,"Gretchen","Kinney","1974-10-18","viole",1),
    (7,"Driscoll","Klein","1970-10-05","blue",5),
    (8,"Karyn","Diaz","1969-02-24","red",1),
    (9,"Merritt","Guy","1974-10-17","indigo",4),
    (10,"Kylan","Sexton","1975-02-28","black",4),
    (11,"Jordan","Estes","1969-12-07","indigo",4),
    (12,"Hope","Coffey","1973-12-24","green",5),
    (13,"Vivian","Crane","1970-08-27","gray",5),
    (14,"Clio","Noel","1972-12-12","red",5),
    (15,"Hope","Silva","1970-07-01","blue",5),
    (16,"Ayanna","Jarvis","1974-02-11","orange",5),
    (17,"Chanda","Boyer","1973-04-01","green",4),
    (18,"Chadwick","Knight","1973-04-29","yellow",1);

## Strings

### LENGTH()

In [10]:
%sql SELECT surname, LENGTH(surname) FROM persons LIMIT 5;

[('Hamilton', 8), ('Holcomb', 7), ('Garrett', 7), ('Fry', 3), ('Conway', 6)]

### UPPER(), LOWER()

In [11]:
%sql SELECT surname, UPPER(surname), LOWER(surname) FROM persons LIMIT 5;

[('Hamilton', 'HAMILTON', 'hamilton'),
 ('Holcomb', 'HOLCOMB', 'holcomb'),
 ('Garrett', 'GARRETT', 'garrett'),
 ('Fry', 'FRY', 'fry'),
 ('Conway', 'CONWAY', 'conway')]

La expresión `BETWEEN` en SQL se utiliza para verificar si un valor se encuentra dentro de un rango de valores.

En este caso, se evalúa si los textos `surname` están entre la **c** y la **k**, pero como los textos `surname` tienen más de una letra ninguno va a cumplir la condición.

In [12]:
#
# Condicional para strings
#
%sql SELECT surname, surname BETWEEN 'c' AND 'k' FROM persons;

[('Hamilton', 0),
 ('Holcomb', 0),
 ('Garrett', 0),
 ('Fry', 0),
 ('Conway', 0),
 ('Kinney', 0),
 ('Klein', 0),
 ('Diaz', 0),
 ('Guy', 0),
 ('Sexton', 0),
 ('Estes', 0),
 ('Coffey', 0),
 ('Crane', 0),
 ('Noel', 0),
 ('Silva', 0),
 ('Jarvis', 0),
 ('Boyer', 0),
 ('Knight', 0)]

In [7]:
%sql SELECT surname, surname BETWEEN 'Aaaaaaaa' AND 'Zzzzzzzz' FROM persons;

[('Hamilton', 1),
 ('Holcomb', 1),
 ('Garrett', 1),
 ('Fry', 1),
 ('Conway', 1),
 ('Kinney', 1),
 ('Klein', 1),
 ('Diaz', 1),
 ('Guy', 1),
 ('Sexton', 1),
 ('Estes', 1),
 ('Coffey', 1),
 ('Crane', 1),
 ('Noel', 1),
 ('Silva', 1),
 ('Jarvis', 1),
 ('Boyer', 1),
 ('Knight', 1)]

### LIKE

La función LIKE permite buscar patrones específicos dentro de cadenas de caracteres.


    > LIKE 'a%'   : Encuentra los valores que inician con 'a'
    > LIKE '%a'   : Encuentra los valores que terminan con 'a'
    > LIKE '%a%'  : Encuentra los valores que contengan 'a' en cualquier posición
    > LIKE '_a%'  : Encuentra los valores que contengan 'a' en la segunda posición
    > LIKE 'a_%_%': Encuentra los valores que inician con 'a' y tienen al menos 3 caracteres
    > LIKE 'a%o'  : Encuentra los valores que inician con 'a' y terminan con 'o'

In [13]:
%sql SELECT * FROM persons WHERE color LIKE 'b%';

[(4, 'Roth', 'Fry', '1975-01-29', 'black', 1),
 (5, 'Zoe', 'Conway', '1974-07-03', 'blue', 2),
 (7, 'Driscoll', 'Klein', '1970-10-05', 'blue', 5),
 (10, 'Kylan', 'Sexton', '1975-02-28', 'black', 4),
 (15, 'Hope', 'Silva', '1970-07-01', 'blue', 5)]

In [14]:
%sql SELECT * FROM persons WHERE color NOT LIKE 'b%';

[(1, 'Vivian', 'Hamilton', '1971-07-08', 'green', 1),
 (2, 'Karen', 'Holcomb', '1974-05-23', 'green', 4),
 (3, 'Cody', 'Garrett', '1973-04-22', 'orange', 1),
 (6, 'Gretchen', 'Kinney', '1974-10-18', 'viole', 1),
 (8, 'Karyn', 'Diaz', '1969-02-24', 'red', 1),
 (9, 'Merritt', 'Guy', '1974-10-17', 'indigo', 4),
 (11, 'Jordan', 'Estes', '1969-12-07', 'indigo', 4),
 (12, 'Hope', 'Coffey', '1973-12-24', 'green', 5),
 (13, 'Vivian', 'Crane', '1970-08-27', 'gray', 5),
 (14, 'Clio', 'Noel', '1972-12-12', 'red', 5),
 (16, 'Ayanna', 'Jarvis', '1974-02-11', 'orange', 5),
 (17, 'Chanda', 'Boyer', '1973-04-01', 'green', 4),
 (18, 'Chadwick', 'Knight', '1973-04-29', 'yellow', 1)]

In [15]:
#
# Conectores lógicos (AND).
#
%sql SELECT * FROM persons WHERE color = 'blue' AND firstname LIKE 'Z%';

[(5, 'Zoe', 'Conway', '1974-07-03', 'blue', 2)]

In [16]:
#
# Conectores lógicos (OR).
#
%sql SELECT * FROM persons WHERE color = 'blue' OR firstname LIKE 'K%';

[(2, 'Karen', 'Holcomb', '1974-05-23', 'green', 4),
 (5, 'Zoe', 'Conway', '1974-07-03', 'blue', 2),
 (7, 'Driscoll', 'Klein', '1970-10-05', 'blue', 5),
 (8, 'Karyn', 'Diaz', '1969-02-24', 'red', 1),
 (10, 'Kylan', 'Sexton', '1975-02-28', 'black', 4),
 (15, 'Hope', 'Silva', '1970-07-01', 'blue', 5)]

In [17]:
#
# Condicional para campos string
#
%sql SELECT * FROM persons WHERE color IN ('blue','black');

[(4, 'Roth', 'Fry', '1975-01-29', 'black', 1),
 (5, 'Zoe', 'Conway', '1974-07-03', 'blue', 2),
 (7, 'Driscoll', 'Klein', '1970-10-05', 'blue', 5),
 (10, 'Kylan', 'Sexton', '1975-02-28', 'black', 4),
 (15, 'Hope', 'Silva', '1970-07-01', 'blue', 5)]

In [18]:
#
# Condicional para campos string
#
%sql SELECT * FROM persons WHERE color NOT IN ('blue','black');

[(1, 'Vivian', 'Hamilton', '1971-07-08', 'green', 1),
 (2, 'Karen', 'Holcomb', '1974-05-23', 'green', 4),
 (3, 'Cody', 'Garrett', '1973-04-22', 'orange', 1),
 (6, 'Gretchen', 'Kinney', '1974-10-18', 'viole', 1),
 (8, 'Karyn', 'Diaz', '1969-02-24', 'red', 1),
 (9, 'Merritt', 'Guy', '1974-10-17', 'indigo', 4),
 (11, 'Jordan', 'Estes', '1969-12-07', 'indigo', 4),
 (12, 'Hope', 'Coffey', '1973-12-24', 'green', 5),
 (13, 'Vivian', 'Crane', '1970-08-27', 'gray', 5),
 (14, 'Clio', 'Noel', '1972-12-12', 'red', 5),
 (16, 'Ayanna', 'Jarvis', '1974-02-11', 'orange', 5),
 (17, 'Chanda', 'Boyer', '1973-04-01', 'green', 4),
 (18, 'Chadwick', 'Knight', '1973-04-29', 'yellow', 1)]

## ORDER BY

In [19]:
%sql SELECT * FROM persons ORDER BY color;

[(4, 'Roth', 'Fry', '1975-01-29', 'black', 1),
 (10, 'Kylan', 'Sexton', '1975-02-28', 'black', 4),
 (5, 'Zoe', 'Conway', '1974-07-03', 'blue', 2),
 (7, 'Driscoll', 'Klein', '1970-10-05', 'blue', 5),
 (15, 'Hope', 'Silva', '1970-07-01', 'blue', 5),
 (13, 'Vivian', 'Crane', '1970-08-27', 'gray', 5),
 (1, 'Vivian', 'Hamilton', '1971-07-08', 'green', 1),
 (2, 'Karen', 'Holcomb', '1974-05-23', 'green', 4),
 (12, 'Hope', 'Coffey', '1973-12-24', 'green', 5),
 (17, 'Chanda', 'Boyer', '1973-04-01', 'green', 4),
 (9, 'Merritt', 'Guy', '1974-10-17', 'indigo', 4),
 (11, 'Jordan', 'Estes', '1969-12-07', 'indigo', 4),
 (3, 'Cody', 'Garrett', '1973-04-22', 'orange', 1),
 (16, 'Ayanna', 'Jarvis', '1974-02-11', 'orange', 5),
 (8, 'Karyn', 'Diaz', '1969-02-24', 'red', 1),
 (14, 'Clio', 'Noel', '1972-12-12', 'red', 5),
 (6, 'Gretchen', 'Kinney', '1974-10-18', 'viole', 1),
 (18, 'Chadwick', 'Knight', '1973-04-29', 'yellow', 1)]

## GROUP BY

### COUNT()

In [20]:
#
# Cuenta cuantos registros hay en la tabla.
#
%sql SELECT COUNT(*) FROM persons;

[(18,)]

### MAX(), MIN(), SUM(), AVG()

In [34]:
%%sqlite3
SELECT 
    color,
    MAX(quantity),
    MIN(quantity),
    SUM(quantity),
    AVG(quantity)
FROM
    persons
GROUP BY
    color;

[('black', 4, 1, 5, 2.5),
 ('blue', 5, 2, 12, 4.0),
 ('gray', 5, 5, 5, 5.0),
 ('green', 5, 1, 14, 3.5),
 ('indigo', 4, 4, 8, 4.0),
 ('orange', 5, 1, 6, 3.0),
 ('red', 5, 1, 6, 3.0),
 ('viole', 1, 1, 1, 1.0),
 ('yellow', 1, 1, 1, 1.0)]

In [22]:
%%sqlite3
SELECT 
    color,
    quantity
FROM
    persons
WHERE
    quantity = (SELECT MIN(quantity) FROM persons);

[('green', 1),
 ('orange', 1),
 ('black', 1),
 ('viole', 1),
 ('red', 1),
 ('yellow', 1)]

---

In [23]:
## Cierre de la conexión
conn.close()